# DataLab

## Koulutus: 2.12.2021

# Aihe: lähimmän naapurin luokittelija


Tässä harjoituksessa tutustumme lähimmän naapurin luokittelijaan (k nearest neighbor classification).

Luokitteluongelma on ohjattua oppimista eli kuuluu ennustavan analytiikan piiriin, eli tarvitsemme syötedatan lisäksi myös vastemuuttujaksi tunnetun luokkkatiedon. Luokitteluongelmissa ennustettava muuttuja eli vastemuuttuja on kategorinen (ei siis jatkuva). 

Scikit-learn -kirjasto sisältää paljon vaihtoehtoja luokitteluun, tänään tutustumme k lähimmän naapurin luokittelijaan. K on luokittelijan paramateri, sillä valitaan kuinka moneen lähimpään naapuriin uutta, luokiteltavaa dataa verrataan. Yksinkertaisimmillaan k = 1 eli annetaan uuden datapisteen luokaksi lähimmän naapurin luokka, joka on siis tunnettu.

In [1]:
# Ladataan peruskirjastot eli Numpy:
import numpy as np

# Visualisointikirjasto Matplotlib ladataan seuraavaksi:
import matplotlib.pyplot as plt

# Ladataan lähimmän naapurin luokitin:
from sklearn import neighbors

# Kirjasto luokittelutarkkuuden arviointiin ja datan opetus/validointijakoon:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Ladataan kirjasto esimerkkidatajoukkojen lataamista varten:
from sklearn import datasets

Ensimmäiseksi käytämme scikit-learn kirjastossa valmiina olevaa datajoukkoa, joka määrittelee luokitteluongelman.

## Lähimmän naapurin luokitin, Iris-datajoukko

Ladataan Iris -datajoukko, tarkempi historia liittyen datajoukkoon löytyy Wikipediasta: https://en.wikipedia.org/wiki/Iris_flower_data_set

In [2]:
# Ladataan iris -datajoukko:
from sklearn.datasets import load_iris

# Palautetaan syötedata ja vastedata erillisiin muuttujiin:
(iris_X, iris_luokka) = load_iris(return_X_y=True)

# Tarkastellaan vielä syötedatan kokoa:
print(iris_X.shape)

# ..ja vastaavasti vastedatan kokoa:
print(len(iris_luokka))

(150, 4)
150


In [3]:
# Luodaan ensin luokittelija:
luokittelija = neighbors.KNeighborsClassifier(n_neighbors=1)

# Opetetaan luokittelija data-aineistolla, oletus on että
# käytetään yhden naapurin luokittelijaa:
luokittelija.fit(iris_X, iris_luokka)

KNeighborsClassifier(n_neighbors=1)

Nyt voimme käyttää luokittelijaa datan luokitteluun. Luokittellaan vaikka aineiston ensimmäinen iris-datajoukon vektori, jossa on 4 elementtiä eli datan dimensio on 4:

In [4]:
# Ensimmäinen data, syötemuuttuja:
eka_X = iris_X[1,:]
print(eka_X)

[4.9 3.  1.4 0.2]


In [5]:
# ensimmäinen data, vastemuuttuja:
eka_luokka = iris_luokka[1]
print(eka_luokka)

0


In [6]:
# Katsotaan mikä luokka saadaan ekalla datalla:
luokittelija.predict([[4.9, 3.0, 1.4, 0.2]])

array([0])

Voimme laskea opetusvirheen luokittelemalla aineiston ja vertailemalla ennustettuja luokkia vastemuuttujiin, koska ne on opetusaineistossa tunnettu.

In [7]:
# Ennustetaan opetusjoukon avulla 
ennustettu_luokka = luokittelija.predict(iris_X)

In [8]:
# ymmärrä miten virhe (tai oikein luokitellut) lasketaan:
oikein_luokitellut = np.sum(ennustettu_luokka == iris_luokka)
print(oikein_luokitellut)

150


In [9]:
# Tai sama oikeinluokittelu prosentteina:
oikein_luokitellut = np.sum(ennustettu_luokka == iris_luokka) /len(iris_luokka)
print(oikein_luokitellut)

1.0


In [10]:
# Tai virhe prosentteina:
väärin_luokitellut = 1 - oikein_luokitellut
print(väärin_luokitellut)

0.0


Onneksi olkoon! Kaikki luokiteltiin oikein! Tai hetkinen..

Miksi oli helppoa saada kaikki oikein opetusjoukossa kun käytettiin 1 lähimmän naapurin luokittelijaa? Onko luokittelutulos ennustettavissa pelkän asetelman perusteella (on, mutta mieti miksi!)

## Ristiinvalidointi luokittelussa, Iris-datajoukko

Nyt jaamme datajoukon opetusjoukkoon ja testijoukkoon, jotta voimme arvioida luokittelijan yleistyskykyä.

In [11]:
iris_x_opetus, iris_x_validointi, iris_luokka_opetus, iris_luokka_validointi = train_test_split(iris_X, iris_luokka, test_size=0.7)

# Luodaan ensin luokittelija, tässä naapureiden määrä k = 1:
luokittelija_val = neighbors.KNeighborsClassifier(n_neighbors=1)

# Opetetaan luokittelija valitulla opetusjoukolla, jätetään validointijoukko
# toistaiseksi käyttämättä:
iris_knn_val = luokittelija.fit(iris_x_opetus, iris_luokka_opetus)

## Arvioidaan virheet opetus ja validointijoukolle:

In [12]:
# Luokitellaan opetusjoukko (joka luokittuu 100% oikein):
ennustus_opetus = luokittelija.predict(iris_x_opetus)

# Luokitellaan validointijoukko, jota ei ole käytetty mallin opetuksessa:
ennustus_validointi = luokittelija.predict(iris_x_validointi)

# Lasketaan opetusjoukon luokittelutarkkuus vertailemalla ennustuksia oikeiden luokkien kanssa:
opetus_tarkkuus = accuracy_score(iris_luokka_opetus, ennustus_opetus)
print("Tarkkuus (opetus):\t{:6.4f}".format(opetus_tarkkuus))

# Lasketaan valindointijoukon luokittelutarkkuus vertailemalla ennustuksia oikeiden luokkien kanssa:
validointi_tarkkuus = accuracy_score(iris_luokka_validointi, ennustus_validointi)
print("Tarkkuus (validointi):\t{:6.4f}".format(validointi_tarkkuus))

Tarkkuus (opetus):	1.0000
Tarkkuus (validointi):	0.9238


Tehtävä 1. Vaihda luokittelijaa 3 lähimään naapurin luokittelijaksi. Katso miten opetusjoukon virhe muuttuu.

Tehtävä 2. Vaihda datajoukkoa luokitteluongelmassa. Tutustu mitä datajoukkoja on saatavilla scikit-learn -kirjastossa tai ota joku luokitteluongelma UCI dataluettelosta, joka löytyy osoitteesta:
https://archive.ics.uci.edu/ml/index.php

Käytä opetuksessa erillisiä opetus- ja validointijoukkoja!